# Piperack Generator

## Module imports and connection to STAAD

In [13]:
from base.helper import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.piperack.portal import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output

add_beams = add_beams(geometry=geometry)

nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None)
beam_nos:list = list(beam_objects.keys())

## Inputs

In [14]:
base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_to_portal_distance = 8
portal_count = 7
columns = [0,width_of_piperack]
tiers = [3,6,9,12,14]
long_beams = [4.5,7.5,10.5,tiers[-1]]

## Generator Code

In [15]:
portals = []
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []

portal = PiperackPortal(base=base_point_of_first_portal) 

for column in columns:
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tiers[-1])) \

for tier in tiers:
    for column_i in range(len(columns)-1):
        portal.add_beam(Beam3D(start=Point3D(columns[column_i],tier,0),end=Point3D(columns[column_i+1],tier,0))) 

portals = [portal.shift(Point3D(0,0,portal_to_portal_distance*i)) for i in range(portal_count)]

for portal_i in range(len(portals)-1):
    for column in columns:
        for long_beam in long_beams:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)

portal_beam_ids = [] 
portal_column_ids = [] 
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []

for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]

long_beam_ids = add_beams(longitudinal_beams)

create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

stubs = beams_sorted_yzx(stubs)

for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        intermediate_transverse_beams.append(Beam3D(start=stub1.mid(),end=stub2.mid()))

stub_ids = add_beams(stubs)
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)

portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for group_y in portal_beams_y.values():
    sorted_beams = beams_sorted_yzx(group_y)
    for beam_i in range(len(sorted_beams)-1):
        beam1 = sorted_beams[beam_i]
        beam2 = sorted_beams[beam_i+1]
        intermediate_long_beams.append(Beam3D(start=beam1.mid(),end=beam2.mid()))


intermediate_long_ids = add_beams(intermediate_long_beams)

markdown_output = ''
markdown_output = "## Members Created\n\n"
markdown_output += "| Member Types | IDs | \n"
markdown_output += "| --- | --- |\n"
markdown_output += f"| Portal Beams | {portal_beam_ids} |\n"
markdown_output += f"| Portal Columns | {portal_column_ids} |\n"
markdown_output += f"| Long Beams | {long_beam_ids} |\n"
markdown_output += f"| Stubs | {stub_ids} |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} |\n"
markdown_output += f"| Intermediate Long Beams | {intermediate_long_ids} |\n"

display(Markdown(markdown_output))
    

## Members Created

| Member Types | IDs | 
| --- | --- |
| Portal Beams | [343, 344, 345, 346, 347, 350, 351, 352, 353, 354, 357, 358, 359, 360, 361, 364, 365, 366, 367, 368, 371, 372, 373, 374, 375, 378, 379, 380, 381, 382, 385, 386, 387, 388, 389] |
| Portal Columns | [348, 349, 355, 356, 362, 363, 369, 370, 376, 377, 383, 384, 390, 391] |
| Long Beams | [392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439] |
| Stubs | [440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475] |
| Intermediate Trans Beams | [226, 229, 232, 235, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268, 271, 274, 277] |
| Intermediate Long Beams | [280, 282, 284, 286, 288, 290, 293, 295, 297, 299, 301, 303, 306, 308, 310, 312, 314, 316, 319, 321, 323, 325, 327, 329, 332, 334, 336, 338, 340, 342] |
